In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pickle
import time
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
def tstr():
    '''
    provides a string of current time in PST
    for time sensitive file names with no colons or spaces
    '''
    out = ''
    for i in time.ctime():
        if i!=' ' and i!=':':
            out+=i
        else:
            out+='_'
    return out

In [7]:
catmednan = np.load('/content/drive/MyDrive/Students/catCirrMedNan.npy')

In [8]:
mednan = np.load('/content/drive/MyDrive/Students/cirrhosis_medNaN.npy')

In [9]:
def gen_model():
    model = keras.Sequential([
            layers.Dense(100, activation="relu", 
                    kernel_regularizer=keras.regularizers.L2(3e-2)),
            layers.Dense(10, activation="relu", 
                    kernel_regularizer=keras.regularizers.L2(3e-2)),
            layers.Dense(4,activation= 'softmax',
                    kernel_regularizer=keras.regularizers.L2(3e-2 ))])
    model.compile(
        loss=keras.losses.CategoricalCrossentropy(),
        optimizer=tf.keras.optimizers.Adam(learning_rate=5e-3),
        metrics=[ 'accuracy'])
    return model

In [10]:
def get_train(x=None, y = None, lst = [1,3,6]):
  outx = []
  outy = []
  for i in lst:
      outx.append(x[i])
      outy.append(y[i])
      
  return np.asarray(outx), np.asarray(outy)
        

In [15]:
def train(train_examples=None,train_labels=None, 
        validation_data = None, model = None, epochs = 2):
  if model==None:
    model = gen_model()
  hist = model.fit(train_examples,train_labels, 
      validation_data = validation_data,epochs = epochs, verbose=1)
  return hist.history['val_loss'][-1]

In [70]:
def tmcDataShapley(x=catmednan[:325,:-4], y = catmednan[:325,-4:],
                   valx = catmednan[325:,:-4], valy= catmednan[325:,-4:], 
                   N = 5, label = ''):
  Ndata, Ddata = x.shape
  results = {}
  for i in range(N):
    delta_loss_lst = [100.0,100.0,100.0]
    loss = 100
    delta_loss=100
    dlosses = []
    losses = []
    lst = []
    
    while np.sum(np.abs(delta_loss_lst)<3) <len(delta_loss_lst) and len(lst)<Ndata:
      # these two conditionals concern whether there has been recent plateau in
      # learning and whether all samples have been used, both cases lead to
      # a breaking from the loop
      model = gen_model()
      k= np.random.randint(0,Ndata)
      while k in lst:
        k= np.random.randint(0,Ndata)
      lst.append(k)
      if len(lst)%5==0:
        print('running with ',len(lst), ' samples')
      newx,newy = get_train(x,y,lst)
      newloss = train(x,y,(valx,valy), model)
      if len(lst)>1:
        if k not in results:
          results[k] = []
        delta_loss = loss-newloss
        if len(dlosses)<4:
          pass
        else:
          delta_loss_lst = dlosses[-3:]

        results[k].append(delta_loss)
      loss = newloss
      losses.append(loss)
      dlosses.append(delta_loss)
      tf.keras.backend.clear_session()
      if len(lst)%20==0:
        print('\nLIST HAS '+str(len(lst))+' SAMPLES\n')
        print('The latest delta loss values are ',str(delta_loss_lst))
        print('\nAnd the latest loss is ',loss,'\n')
    # results['dlosses'+str(i)] = dlosses
    # results['losses'+str(i)] = losses
    # np.save("/content/drive/MyDrive/Students/shapley/David_"
    #         +label+".pkl",results)
    meanresults = {}
    for pnt in results:
      meanresults[pnt] = [np.mean(results[pnt]),len(results[pnt])]
    df = pd.DataFrame(meanresults,index = ['value','samples'])
    df.to_csv('/content/drive/MyDrive/Students/shapley/David_'+label+'.csv')
    # pickle.dump(dict,f)
    # f.close()

    print('\nSAVED RESULTS for loop\n'+str(i))
  return results

In [72]:
results = tmcDataShapley(label='DicTests2',N=7)
##Put YOUR NAME IN THE LABEL AND A WORD OR TWO WITH DETAIL ABOUT EXPERIMENTS

Epoch 1/2
11/11 [==============================] - 1s 22ms/step - loss: 292.2105 - accuracy: 0.3262 - val_loss: 7.6485 - val_accuracy: 0.3448
Epoch 2/2
11/11 [==============================] - 0s 4ms/step - loss: 17.4361 - accuracy: 0.3538 - val_loss: 2.2822 - val_accuracy: 0.3793
Epoch 1/2
11/11 [==============================] - 1s 20ms/step - loss: 458.3246 - accuracy: 0.3200 - val_loss: 232.5539 - val_accuracy: 0.3448
Epoch 2/2
11/11 [==============================] - 0s 6ms/step - loss: 90.6867 - accuracy: 0.3754 - val_loss: 34.0957 - val_accuracy: 0.1379
Epoch 1/2
11/11 [==============================] - 1s 21ms/step - loss: 432.6791 - accuracy: 0.3354 - val_loss: 232.1774 - val_accuracy: 0.2644
Epoch 2/2
11/11 [==============================] - 0s 6ms/step - loss: 194.2318 - accuracy: 0.3969 - val_loss: 166.8059 - val_accuracy: 0.2989
Epoch 1/2
11/11 [==============================] - 1s 20ms/step - loss: 139.9004 - accuracy: 0.2769 - val_loss: 2.7698 - val_accuracy: 0.2184
Epoc

In [56]:
results

{6: [139.54272079467773],
 13: [27.26806640625],
 16: [158.43497610092163],
 26: [40.93973684310913],
 27: [-47.563464879989624],
 28: [-40.720510959625244],
 36: [-0.026249408721923828],
 38: [225.93558502197266],
 39: [83.73796391487122],
 43: [-126.24992370605469],
 45: [178.80688858032227],
 46: [-13.515045642852783],
 53: [69.3923089504242],
 56: [131.60641598701477],
 71: [45.71794652938843, 0.008728981018066406],
 77: [248.81498646736145],
 79: [-0.0700981616973877],
 81: [47.182281494140625, -0.038187265396118164],
 83: [88.85386991500854],
 88: [33.436614990234375],
 113: [0.07014322280883789],
 114: [95.13594436645508],
 121: [-227.05224609375],
 128: [-0.12750744819641113],
 129: [136.9308798313141],
 130: [115.89377212524414],
 133: [0.006127119064331055],
 136: [0.25372767448425293],
 137: [597.5738425254822],
 139: [-0.19249486923217773],
 140: [95.75730419158936],
 144: [-252.06532216072083],
 148: [252.23107433319092],
 153: [-27.73117208480835],
 157: [-248.98738312721